In [15]:
import tensorflow as tf
import numpy as np

In [22]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
W = tf.Variable(tf.zeros(shape=[1,2]))

In [ ]:
Variable:
故名思议，是变量的意思。一般用来表示图中的各计算参数，包括矩阵，向量等。例如，我要表示上图中的模型，那表达式就是 
y=Relu(Wx+b)

这里W和b是我要用来训练的参数，那么此时这两个值就可以用Variable来表示。Variable的初始函数有很多其他选项，这里先不提，只输入
一个Tensor也是可以的
注意，此时W一样是一个抽象的概念，而且与Tensor不同，Variable必须初始化以后才有具体的值

In [6]:
tensor = tf.zeros(shape=[1,2])
variable = tf.Variable(tensor)
sess = tf.InteractiveSession()
# print(sess.run(variable))  # 会报错
sess.run(tf.global_variables_initializer()) # 对variable进行初始化
print(sess.run(variable))
#===>[[ 0.  0.]]

[[ 0.  0.]]


In [ ]:
WARNING:tensorflow:From /Users/michael/.pyenv/versions/3.5.2/lib/python3.5/site-packages/tensorflow/python/util/tf_should_use.py:170: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 0.  0.]]
说明tensorflow 在不同的版本上会有一些不同

In [ ]:
placeholder
又叫占位符，同样是一个抽象的概念。用于表示输入输出数据的格式。告诉系统：这里有一个值/向量/矩阵，现在我没法给你具体数值，
不过我正式运行的时候会补上的！例如上式中的x和y。因为没有具体数值，所以只要指定尺寸即可

In [7]:
x = tf.placeholder(tf.float32,[1, 5],name='input')
y = tf.placeholder(tf.float32,[None, 5],name='input')

In [8]:
print(type(x),type(y))

<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>


In [10]:
print(x),print(y)

Tensor("input:0", shape=(1, 5), dtype=float32)
Tensor("input_1:0", shape=(?, 5), dtype=float32)


(None, None)

In [13]:
#上面有两种形式，第一种x，表示输入是一个[1,5]的横向量。 
#而第二种形式，表示输入是一个[?,5]的矩阵。那么什么情况下会这么用呢?就是需要输入一批[1,5]的数据的时候。
#比如我有一批共10个数据，那我可以表示成[10,5]的矩阵。
#如果是一批5个，那就是[5,5]的矩阵。tensorflow会自动进行批处理

In [16]:
a = np.arange(0,51,10).reshape(6,1) + np.arange(6)

In [17]:
print(a)

[[ 0  1  2  3  4  5]
 [10 11 12 13 14 15]
 [20 21 22 23 24 25]
 [30 31 32 33 34 35]
 [40 41 42 43 44 45]
 [50 51 52 53 54 55]]


In [18]:
# 建立抽象模型
x = tf.placeholder(tf.float32, [None, 784]) # 输入占位符
y = tf.placeholder(tf.float32, [None, 10])  # 输出占位符（预期输出）
W = tf.Variable(tf.zeros([784, 10]))        
b = tf.Variable(tf.zeros([10]))
a = tf.nn.softmax(tf.matmul(x, W) + b)      # a表示模型的实际输出

# 定义损失函数和训练方法
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(a), reduction_indices=[1])) # 损失函数为交叉熵
optimizer = tf.train.GradientDescentOptimizer(0.5) # 梯度下降法，学习速率为0.5
train = optimizer.minimize(cross_entropy)  # 训练目标：最小化损失函数

In [19]:
correct_prediction = tf.equal(tf.argmax(a, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('data_dir', '/tmp/data/', 'Directory for storing data') # 把数据放在/tmp/data文件夹中

In [25]:

mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)   # 读取数据集

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [27]:
sess = tf.InteractiveSession()      # 建立交互式会话
tf.initialize_all_variables().run() # 所有变量初始化
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)    # 获得一批100个数据
    train.run({x: batch_xs, y: batch_ys})   # 给训练模型提供输入和输出
print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.9106
